# Zusammenfügen von Datensätzen in `pandas`

Wir kennen in Pandas nun schon `pd.concat`. Diese Funktion dient zum 
*einfachen* Aneinanderfügen von Tabellen, ohne Berücksichtigung von Spalten,
 die diese gemeinsam haben.

Heute lernen wir, wie wir Tabellen basierend auf übereinstimmenden Spalten 
(z.B. Bestell-ID; Modellnummer; ...) zusammenfügen. Dabei werden anhand der
 Indizes oder anhand einer gemeinsamen Spalte Einträge verbunden, die in 
 beiden Tabellen übereinstimmen. Wir kennen diese "Joins" noch aus Excel 
 mit den Funktionen `SVERWEIS()` und `INDEX(VERGLEICH())` und aus Power 
 Query. In Pandas benutzt man mit die DataFrame-Methoden `join()` und `merge()`.

Hierfür ist es nützlich, sich noch einmal die verschiedenen Arten anzuschauen, auf
  die man Tabellen zusammenfügen kann – die sogenannten Joins.

In [1]:
import pandas as pd


## Anfügen von Daten: pd.concat


In [2]:
# Series mit Temperatur-Messwerten:
data = [4.5, 6.3, 3.8, 5.1, 4.9, 5.7, 4.2, 6.0]
temp_series = pd.Series(data, name='Temperatur')
temp_series

0    4.5
1    6.3
2    3.8
3    5.1
4    4.9
5    5.7
6    4.2
7    6.0
Name: Temperatur, dtype: float64

In [3]:
# Series mit Zeitstempeln für Messzeitpunkt:
uhrzeiten = ['2023-01-02 19:08',
             '2023-01-04 18:17',
             '2023-01-06 06:03',
             '2023-01-09 02:17',
             '2023-01-12 22:02',
             '2023-01-17 16:00',
             '2023-01-22 21:04',
             '2023-01-24 11:16']

# Eigentlich besser, man macht datetime-Objekte daraus, aber das Thema kommt erst später dran,
# bitte also Geduld ;)

# So könnte man das tun: pd.Series(pd.to_datetime(uhrzeiten))

time_series = pd.Series(uhrzeiten, name='Zeitstempel')
time_series

0    2023-01-02 19:08
1    2023-01-04 18:17
2    2023-01-06 06:03
3    2023-01-09 02:17
4    2023-01-12 22:02
5    2023-01-17 16:00
6    2023-01-22 21:04
7    2023-01-24 11:16
Name: Zeitstempel, dtype: object

#### Beide Serien zu einem DataFrame verbinden


In [5]:
# temp_series und time_series verbinden
temp_df = pd.concat([time_series, temp_series], axis=1)
temp_df

,Zeitstempel,Temperatur
0,2023-01-02 19:08,4.5
1,2023-01-04 18:17,6.3
2,2023-01-06 06:03,3.8
3,2023-01-09 02:17,5.1
4,2023-01-12 22:02,4.9
5,2023-01-17 16:00,5.7
6,2023-01-22 21:04,4.2
7,2023-01-24 11:16,6.0


#### Was, wenn die Indices nicht so gut zusammenspielen?

In [7]:
# Eigene Indizes vergeben:

time_series_2 = time_series.copy()
# Lücken: 0, 1, 4, 10
time_series_2.index = [2, 3, 5, 6, 7, 8, 9, 11]

temp_series_2 = temp_series.copy()
# Lücken: 1, 3, 9, 11
temp_series_2.index = [0, 2, 4, 5, 6, 7, 8, 10]

print(time_series_2)
print()
print(temp_series_2)

2     2023-01-02 19:08
3     2023-01-04 18:17
5     2023-01-06 06:03
6     2023-01-09 02:17
7     2023-01-12 22:02
8     2023-01-17 16:00
9     2023-01-22 21:04
11    2023-01-24 11:16
Name: Zeitstempel, dtype: object

0     4.5
2     6.3
4     3.8
5     5.1
6     4.9
7     5.7
8     4.2
10    6.0
Name: Temperatur, dtype: float64


In [15]:
# Serien mit benannten indizes verbinden
# Erzeugt NaN, wenn Indices nicht in beiden Serien vorkommen
# Standard ist outer Join
temps_df2 = pd.concat([time_series_2, temp_series_2], axis=1)
temps_df2.sort_index()

,Zeitstempel,Temperatur
0,NaN,4.5
2,2023-01-02 19:08,6.3
3,2023-01-04 18:17,NaN
4,NaN,3.8
5,2023-01-06 06:03,5.1
6,2023-01-09 02:17,4.9
7,2023-01-12 22:02,5.7
8,2023-01-17 16:00,4.2
9,2023-01-22 21:04,NaN
10,NaN,6.0


In [20]:
# Wenn es ohne NaNs sein soll:
temps_df2 = pd.concat([time_series_2, temp_series_2], axis=1, join='inner')
temps_df2.sort_index()

,Zeitstempel,Temperatur
2,2023-01-02 19:08,6.3
5,2023-01-06 06:03,5.1
6,2023-01-09 02:17,4.9
7,2023-01-12 22:02,5.7
8,2023-01-17 16:00,4.2


In [ ]:
# Index muss in beiden Serien einzigartig sein (Keine Duplikate)!
# Sonst funktioniert concat nicht.

In [17]:
# Test
temp_series_3 = temp_series.copy()
temp_series_3.index = [2, 3, 5, 6, 7, 7, 9, 11]

time_series_3 = time_series.copy()
time_series_3.index = [0, 2, 2, 5, 6, 7, 8, 10]

In [18]:
temps_df3 = pd.concat([time_series_3, temp_series_3],
                       axis=1)

temps_df3.sort_index()

/tmp/ipykernel_10897/3027033371.py:1: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  temps_df3 = pd.concat([time_series_3, temp_series_3],


ValueError: cannot reindex on an axis with duplicate labels

## Verbinden über Index-Vergleich


### `DataFrame.join`

Verwendet den Index oder eine bestimmte Spalte des DataFrames, der die Methode aufruft und fügt die Daten übereinstimmender Indizes des anderen DataFrames seitlich an. 

Für weitere Infos: [Link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html)

In [21]:
# Erstellen zweier DataFrames mit
# unterschiedlichen Indices
contacts1 = pd.DataFrame({"Name": ["Franz", "Lena", "Chloé"],
                          "Alter": ["67", "31", "41"]},
                          index=["K0", "K1", "K2"])

contacts2 = pd.DataFrame({"Wohnort": ["Rostock", "Nürnberg", "Berlin"],
                          "Telefonnummer": ["030 215783", "030 847735", "030 781404"]},
                          index=["K0", "K2", "K3"])

print(contacts1)
print()
print(contacts2)
# Was stellen wir an den DataFrames fest?

     Name Alter
K0  Franz    67
K1   Lena    31
K2  Chloé    41

     Wohnort Telefonnummer
K0   Rostock    030 215783
K2  Nürnberg    030 847735
K3    Berlin    030 781404


In [22]:
# pd.concat kann zwar auch joins, aber nur inner 
# oder outer join (kein left oder right join)
# Standardverhalten ist übrigens: outer und erzeugt potentiell NaNs:
pd.concat([contacts1, contacts2], axis=1, join='outer')

,Name,Alter,Wohnort,Telefonnummer
K0,Franz,67,Rostock,030 215783
K1,Lena,31,NaN,NaN
K2,Chloé,41,Nürnberg,030 847735
K3,NaN,NaN,Berlin,030 781404


In [23]:
pd.concat([contacts1, contacts2], axis=1, join='inner')

,Name,Alter,Wohnort,Telefonnummer
K0,Franz,67,Rostock,030 215783
K2,Chloé,41,Nürnberg,030 847735


#### DataFrame.join() ermöglicht weitere Joins

In [26]:
# Left Join (Standardverhalten von join!)
# Alle Keys aus dem ERSTEN (linken) Datensatz werden genutzt
# und um Daten aus dem andern (rechten) Datensatz ergänzt.
# An Indexpositionen, über die der rechte Datensatz nicht verfügt, entstehen NaNs:
contacts1.join(contacts2)

,Name,Alter,Wohnort,Telefonnummer
K0,Franz,67,Rostock,030 215783
K1,Lena,31,NaN,NaN
K2,Chloé,41,Nürnberg,030 847735


In [27]:
# Von contacts2 kommend entstehen die Lücken an anderen Stellen, wo eben contacts1 keine Indices hat:
contacts2.join(contacts1)

,Wohnort,Telefonnummer,Name,Alter
K0,Rostock,030 215783,Franz,67
K2,Nürnberg,030 847735,Chloé,41
K3,Berlin,030 781404,NaN,NaN


In [28]:
# Right join
# Keys der rechten (zweiten) Datensatzes werden genutzt,
# und um entsprechende Daten aus dem linken (ersten) ergänzt.
# Wo der erste keine Indices hat, entstehen NaNs 
contacts1.join(contacts2, how='right')

,Name,Alter,Wohnort,Telefonnummer
K0,Franz,67,Rostock,030 215783
K2,Chloé,41,Nürnberg,030 847735
K3,NaN,NaN,Berlin,030 781404


In [29]:
# Outer join
# Alle Keys aus BEIDEN Datensätzen werden genutzt
# Maximale "NaN-Dichte" wird erreicht:
contacts1.join(contacts2, how='outer')

,Name,Alter,Wohnort,Telefonnummer
K0,Franz,67,Rostock,030 215783
K1,Lena,31,NaN,NaN
K2,Chloé,41,Nürnberg,030 847735
K3,NaN,NaN,Berlin,030 781404


In [30]:
# Inner join
# Nur Keys, die in BEIDEN Datensätzen vorhanden sind
# Es kommt zu keinen NaN-Werten (ist unmöglich!):
contacts1.join(contacts2, how='inner')

,Name,Alter,Wohnort,Telefonnummer
K0,Franz,67,Rostock,030 215783
K2,Chloé,41,Nürnberg,030 847735


In [31]:
# Cross join
# Erzeugt eine Kombination jeder Zeile des ersten Datensatzes
# mit jeder Zeile des zweiten Datensatzes (hier nicht gerade sinnvoll)
contacts1.join(contacts2, how='cross')

,Name,Alter,Wohnort,Telefonnummer
0,Franz,67,Rostock,030 215783
1,Franz,67,Nürnberg,030 847735
2,Franz,67,Berlin,030 781404
3,Lena,31,Rostock,030 215783
4,Lena,31,Nürnberg,030 847735
5,Lena,31,Berlin,030 781404
6,Chloé,41,Rostock,030 215783
7,Chloé,41,Nürnberg,030 847735
8,Chloé,41,Berlin,030 781404


#### Zusammenfügen mehrerer Datensätze

In [32]:
print(contacts1)
print()
print(contacts2)

     Name Alter
K0  Franz    67
K1   Lena    31
K2  Chloé    41

     Wohnort Telefonnummer
K0   Rostock    030 215783
K2  Nürnberg    030 847735
K3    Berlin    030 781404


In [33]:
# Mit zwei dfs kennen wir das Spiel schon:
contacts1.join(contacts2, how='outer')

,Name,Alter,Wohnort,Telefonnummer
K0,Franz,67,Rostock,030 215783
K1,Lena,31,NaN,NaN
K2,Chloé,41,Nürnberg,030 847735
K3,NaN,NaN,Berlin,030 781404


In [34]:
# Aber jetzt haben wir noch eine Nummer 3:
contacts3 = pd.DataFrame({"Position": ["Rentner", "Verkäuferin", "Data Engineer"],
                    "Gehalt": ["1400", "3000", "3800"]},
                   index=["K1", "K3", "K4"])

contacts3

,Position,Gehalt
K1,Rentner,1400
K3,Verkäuferin,3000
K4,Data Engineer,3800


In [37]:
print(contacts1)
print(contacts2)
print(contacts3)

     Name Alter
K0  Franz    67
K1   Lena    31
K2  Chloé    41
     Wohnort Telefonnummer
K0   Rostock    030 215783
K2  Nürnberg    030 847735
K3    Berlin    030 781404
         Position Gehalt
K1        Rentner   1400
K3    Verkäuferin   3000
K4  Data Engineer   3800


In [35]:
# Joinen mehrerer dfs an contacts1 über Liste möglich:
contacts1.join([contacts2, contacts3], how='outer')

,Name,Alter,Wohnort,Telefonnummer,Position,Gehalt
K0,Franz,67,Rostock,030 215783,NaN,NaN
K1,Lena,31,NaN,NaN,Rentner,1400
K2,Chloé,41,Nürnberg,030 847735,NaN,NaN
K3,NaN,NaN,Berlin,030 781404,Verkäuferin,3000
K4,NaN,NaN,NaN,NaN,Data Engineer,3800


In [ ]:
# Bonusfrage: Was zum Teufel ist denn hier passiert?
contacts1.join([contacts2, contacts3], how='inner')
# Es gibt keinen einzigen Index, den beide Tabellen gemein hätten.

,Name,Alter,Wohnort,Telefonnummer,Position,Gehalt


In [39]:
contacts1

,Name,Alter
K0,Franz,67
K1,Lena,31
K2,Chloé,41


In [40]:
# Bonus-Info!
# join hat noch eine weitere Fähigkeit mit 'on'
# Der join kann über eine wählbare Spalte aus dem linken DataFrame 
# mit dem Index des rechten DataFrames erfolgen.
# Wir modifizieren contacts1, sodass dort die "Indices" in einer Spalte vorkommen!

contacts1 = pd.DataFrame({"Name": ["Franz", "Lena", "Chloé"],
                          "Alter": ["67", "31", "41"],
                          "Kontakt-ID": ["K0", "K1", "K2"]})

contacts2 = pd.DataFrame({"Wohnort": ["Rostock", "Nürnberg", "Berlin"],
                          "Telefonnummer": ["030 215783", "030 847735", "030 781404"]}, 
                          index=["K0", "K2", "K3"])

print(contacts1)
print()
print(contacts2)

    Name Alter Kontakt-ID
0  Franz    67         K0
1   Lena    31         K1
2  Chloé    41         K2

     Wohnort Telefonnummer
K0   Rostock    030 215783
K2  Nürnberg    030 847735
K3    Berlin    030 781404


In [41]:
contacts1.join(contacts2, on='Kontakt-ID')

,Name,Alter,Kontakt-ID,Wohnort,Telefonnummer
0,Franz,67,K0,Rostock,030 215783
1,Lena,31,K1,NaN,NaN
2,Chloé,41,K2,Nürnberg,030 847735


In [42]:
# Das geht mit join nicht mehr:
contacts1 = pd.DataFrame({"Name": ["Franz", "Lena", "Chloé"],
                          "Alter": ["67", "31", "41"],
                          "Kontakt-ID": ["K0", "K1", "K2"]})

contacts2 = pd.DataFrame({"Wohnort": ["Rostock", "Nürnberg", "Berlin"],
                          "Telefonnummer": ["030 215783", "030 847735", "030 781404"], 
                          "Kontakt-ID": ["K0", "K2", "K3"]})

print(contacts1)
print()
print(contacts2)

    Name Alter Kontakt-ID
0  Franz    67         K0
1   Lena    31         K1
2  Chloé    41         K2

    Wohnort Telefonnummer Kontakt-ID
0   Rostock    030 215783         K0
1  Nürnberg    030 847735         K2
2    Berlin    030 781404         K3


In [ ]:
contacts1.join(contacts2, on='Kontakt-ID')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
# Aber man kann solche Dinge (und noch mehr) auch mit merge erreichen!

#### Beide Serien zu einem DataFrame verbinden


#### Übungsaufgabe `concat` + `join`

Zeit: 30 Minuten

Gegeben sind Temperaturmessdaten (`temp`), Zeitstempel (`uhrzeiten`), 
 Luftdruckdaten (`druck_dict`) und Geolokationsdaten (`geo_dict`).
1. Wandel die Temperaturdaten und Zeitstempel in Series um und kombiniere 
sie anschließend zu einem DataFrame namens `temp_df`.
2. Wandel die beiden Dictionaries jedes in jeweils einen DataFrame um (`druck_df`, `geo_df`).
3. Füge die Druckdaten an den DataFrame aus 1. an und speichere den neuen 
DataFrame als `df_gesamt`.
4. Kombiniere `df_gesamt` so mit dem Geolokalisation-DataFrame, dass du für 
jede in `df_gesamt` vorkommende Stadt die Breiten- und Längengrade im 
resultierenden DataFrame erhältst. (Tipp: Hierfür müssen die Indices 
verändert werden).

    Output:
    ```
                        Zeitstempel  Temperatur  Luftdruck  Breitengrad  Laengengrad  
    Location                                                              
    Berlin         2023-01-01 19:08         4.5     1001.2        52.31        13.24
    München        2023-01-01 18:17         6.3      997.8        48.80        11.34
    Wilhelmshaven  2023-01-01 06:03         3.8     1002.5          NaN          NaN
    Kassel         2023-01-01 02:17         5.1     1000.1        49.28      -123.13
    Frankfurt      2023-01-01 22:02         4.9      998.9        47.61      -122.33
    Duisburg       2023-01-01 16:00         5.7     1001.5        53.55      -113.49
    Dresden        2023-01-01 21:04         4.2      999.2          NaN          NaN
    Würzburg       2023-01-01 11:16         6.0     1002.8        51.05      -114.07
    ```


In [47]:
# Temperatur-Messwerte
temp = [4.5, 6.3, 3.8, 5.1, 4.9, 5.7, 4.2, 6.0]

# Zeitstempel für Messzeitpunkt
uhrzeiten = ['2023-01-01 19:08',
             '2023-01-01 18:17',
             '2023-01-01 06:03',
             '2023-01-01 02:17',
             '2023-01-01 22:02',
             '2023-01-01 16:00',
             '2023-01-01 21:04',
             '2023-01-01 11:16']

# Orte und Luftdruckmessung
druck_dict = {'Location': ['Berlin', 'München', 'Wilhelmshaven',
                           'Kassel', 'Frankfurt', 'Duisburg',
                           'Dresden', 'Würzburg'],
              'Luftdruck': [1001.2, 997.8, 1002.5, 1000.1, 998.9,
                            1001.5, 999.2, 1002.8]}

# Längen- und Breitengrade der Orte
geo_dict = {'Location': ['Berlin', 'München', 'Hamburg', 'Köln',
                         'Frankfurt', 'Duisburg', 'Kassel', 'Würzburg'],
            'Breitengrad': [52.31, 48.8, 53.33, 45.75, 47.61,
                            53.55, 49.28, 51.05],
            'Laengengrad': [13.24, 11.34, 10.0, -122.43, -122.33,
                            -113.49, -123.13, -114.07]}

In [48]:
# 1.
# Erstellen von Series aus den Daten
temp_series = pd.Series(temp, name='Temperatur')
print(temp_series)

time_series = pd.Series(uhrzeiten, name='Zeitstempel')
print(time_series)

# temp_series und time_series zu df verbinden
temp_df = pd.concat([time_series, temp_series], axis=1)
temp_df

0    4.5
1    6.3
2    3.8
3    5.1
4    4.9
5    5.7
6    4.2
7    6.0
Name: Temperatur, dtype: float64
0    2023-01-01 19:08
1    2023-01-01 18:17
2    2023-01-01 06:03
3    2023-01-01 02:17
4    2023-01-01 22:02
5    2023-01-01 16:00
6    2023-01-01 21:04
7    2023-01-01 11:16
Name: Zeitstempel, dtype: object


,Zeitstempel,Temperatur
0,2023-01-01 19:08,4.5
1,2023-01-01 18:17,6.3
2,2023-01-01 06:03,3.8
3,2023-01-01 02:17,5.1
4,2023-01-01 22:02,4.9
5,2023-01-01 16:00,5.7
6,2023-01-01 21:04,4.2
7,2023-01-01 11:16,6.0


In [49]:
# 2.
# DataFrame aus Luftdruck-Messwerten erzeugen
druck_df = pd.DataFrame(druck_dict)

# DataFrame aus Geodaten erzeugen
geo_df = pd.DataFrame(geo_dict)

In [50]:
# 3.
# Kombinieren von temp_df und druck_df
df_gesamt = pd.concat([temp_df, druck_df], axis=1)
df_gesamt

,Zeitstempel,Temperatur,Location,Luftdruck
0,2023-01-01 19:08,4.5,Berlin,1001.2
1,2023-01-01 18:17,6.3,München,997.8
2,2023-01-01 06:03,3.8,Wilhelmshaven,1002.5
3,2023-01-01 02:17,5.1,Kassel,1000.1
4,2023-01-01 22:02,4.9,Frankfurt,998.9
5,2023-01-01 16:00,5.7,Duisburg,1001.5
6,2023-01-01 21:04,4.2,Dresden,999.2
7,2023-01-01 11:16,6.0,Würzburg,1002.8


In [51]:
# 4.
# Index der dfs auf die Spalte Location setzen
geo_df.set_index("Location", inplace=True)

In [52]:
# Left join des df df_gesamt und geo_df
df_gesamt = df_gesamt.join(geo_df, on='Location')
df_gesamt

,Zeitstempel,Temperatur,Location,Luftdruck,Breitengrad,Laengengrad
0,2023-01-01 19:08,4.5,Berlin,1001.2,52.31,13.24
1,2023-01-01 18:17,6.3,München,997.8,48.80,11.34
2,2023-01-01 06:03,3.8,Wilhelmshaven,1002.5,NaN,NaN
3,2023-01-01 02:17,5.1,Kassel,1000.1,49.28,-123.13
4,2023-01-01 22:02,4.9,Frankfurt,998.9,47.61,-122.33
5,2023-01-01 16:00,5.7,Duisburg,1001.5,53.55,-113.49
6,2023-01-01 21:04,4.2,Dresden,999.2,NaN,NaN
7,2023-01-01 11:16,6.0,Würzburg,1002.8,51.05,-114.07


In [44]:
# Alternative
# 4.
# Index der beiden dfs auf die Spalte Location setzen
df_gesamt.set_index("Location", inplace=True)
geo_df.set_index("Location", inplace=True)

df_gesamt

NameError: name 'df_gesamt' is not defined

In [ ]:
# Alternative
# Left join des df df_gesamt und geo_df
df_gesamt = df_gesamt.join(geo_df)
df_gesamt

,Zeitstempel,Temperatur,Luftdruck,Breitengrad,Laengengrad
Location,,,,,
Berlin,2023-01-01 19:08,4.5,1001.2,52.31,13.24
München,2023-01-01 18:17,6.3,997.8,48.80,11.34
Wilhelmshaven,2023-01-01 06:03,3.8,1002.5,NaN,NaN
Kassel,2023-01-01 02:17,5.1,1000.1,49.28,-123.13
Frankfurt,2023-01-01 22:02,4.9,998.9,47.61,-122.33
Duisburg,2023-01-01 16:00,5.7,1001.5,53.55,-113.49
Dresden,2023-01-01 21:04,4.2,999.2,NaN,NaN
Würzburg,2023-01-01 11:16,6.0,1002.8,51.05,-114.07


In [ ]:
# ENDE ÜBUNG

### `pandas.merge` (Datensätze zusammenfügen über Index oder / und Column)

Bei `pd.merge` können wir ein `on=` Keyword angeben, wodurch wir Tabellen 
auch über normale Spalten statt über den Index zusammenführen können. Hier 
müssen nicht einmal die Spaltennamen zwingend übereinstimmen. Außerdem hat 
`merge` noch viele andere zusätzliche Optionen, die es bei `join` nicht 
gibt, zum Beispiel die Benutzung mehrerer Schlüsselspalten.

Es gibt sogar eine `merge_asof` Funktion, welche auch ungenaue 
Übereinstimmungen erlaubt, ähnlich wie der optionale Parameter 
Bereich_Verweis in Excels SVERWEIS, wo eine ungenaue Übereinstimmung über 
"WAHR" festgelegt werden konnte.
Jedoch gibt es auch hier in Pandas wieder viel mehr Einstellungsmöglichkeiten.

Mehr Information: [Link](https://pandas.pydata.org/docs/reference/api/pandas.merge.html)

In [53]:
# DataFrame aus Übung erstellen vor dem Zusammenführen:
df = pd.concat([time_series, temp_series, druck_df],
               axis=1)
df

,Zeitstempel,Temperatur,Location,Luftdruck
0,2023-01-01 19:08,4.5,Berlin,1001.2
1,2023-01-01 18:17,6.3,München,997.8
2,2023-01-01 06:03,3.8,Wilhelmshaven,1002.5
3,2023-01-01 02:17,5.1,Kassel,1000.1
4,2023-01-01 22:02,4.9,Frankfurt,998.9
5,2023-01-01 16:00,5.7,Duisburg,1001.5
6,2023-01-01 21:04,4.2,Dresden,999.2
7,2023-01-01 11:16,6.0,Würzburg,1002.8


In [54]:
# Geo-Dataframe soll gänzlich anderen Index haben.
# Vorarbeit:
geo_length = len(geo_dict['Location'])

In [55]:
geo_df = pd.DataFrame(geo_dict, index=[f'Eintrag {i}' for i in range (geo_length)])
geo_df

,Location,Breitengrad,Laengengrad
Eintrag 0,Berlin,52.31,13.24
Eintrag 1,München,48.80,11.34
Eintrag 2,Hamburg,53.33,10.00
Eintrag 3,Köln,45.75,-122.43
Eintrag 4,Frankfurt,47.61,-122.33
Eintrag 5,Duisburg,53.55,-113.49
Eintrag 6,Kassel,49.28,-123.13
Eintrag 7,Würzburg,51.05,-114.07


In [58]:
# Zusammenführen beider dfs jetzt durch merge,
# OHNE dass Indices passen:
df.merge(geo_df, on='Location')
# how ist standardmäßig auf 'inner' gesetzt

,Zeitstempel,Temperatur,Location,Luftdruck,Breitengrad,Laengengrad
0,2023-01-01 19:08,4.5,Berlin,1001.2,52.31,13.24
1,2023-01-01 18:17,6.3,München,997.8,48.80,11.34
2,2023-01-01 02:17,5.1,Kassel,1000.1,49.28,-123.13
3,2023-01-01 22:02,4.9,Frankfurt,998.9,47.61,-122.33
4,2023-01-01 16:00,5.7,Duisburg,1001.5,53.55,-113.49
5,2023-01-01 11:16,6.0,Würzburg,1002.8,51.05,-114.07


In [59]:
# Achtung: andere defaults bei .merge() als bei .join()
# bei .merge() ist inner join default (bei .join() ist es left)
df.merge(geo_df, on='Location', how='left')

,Zeitstempel,Temperatur,Location,Luftdruck,Breitengrad,Laengengrad
0,2023-01-01 19:08,4.5,Berlin,1001.2,52.31,13.24
1,2023-01-01 18:17,6.3,München,997.8,48.80,11.34
2,2023-01-01 06:03,3.8,Wilhelmshaven,1002.5,NaN,NaN
3,2023-01-01 02:17,5.1,Kassel,1000.1,49.28,-123.13
4,2023-01-01 22:02,4.9,Frankfurt,998.9,47.61,-122.33
5,2023-01-01 16:00,5.7,Duisburg,1001.5,53.55,-113.49
6,2023-01-01 21:04,4.2,Dresden,999.2,NaN,NaN
7,2023-01-01 11:16,6.0,Würzburg,1002.8,51.05,-114.07


In [60]:
df.merge(geo_df, on='Location', how='outer')

,Zeitstempel,Temperatur,Location,Luftdruck,Breitengrad,Laengengrad
0,2023-01-01 19:08,4.5,Berlin,1001.2,52.31,13.24
1,2023-01-01 18:17,6.3,München,997.8,48.80,11.34
2,2023-01-01 06:03,3.8,Wilhelmshaven,1002.5,NaN,NaN
3,2023-01-01 02:17,5.1,Kassel,1000.1,49.28,-123.13
4,2023-01-01 22:02,4.9,Frankfurt,998.9,47.61,-122.33
5,2023-01-01 16:00,5.7,Duisburg,1001.5,53.55,-113.49
6,2023-01-01 21:04,4.2,Dresden,999.2,NaN,NaN
7,2023-01-01 11:16,6.0,Würzburg,1002.8,51.05,-114.07
8,NaN,NaN,Hamburg,NaN,53.33,10.00
9,NaN,NaN,Köln,NaN,45.75,-122.43


#### merge() erlaubt uns auch das Verbinden von unterschiedlich bezeichneten Spalten

In [61]:
# Umbennenen der Spalte Location von geo_df in Stadt
geo_df.rename(columns={'Location': 'City'}, inplace=True)
geo_df

,City,Breitengrad,Laengengrad
Eintrag 0,Berlin,52.31,13.24
Eintrag 1,München,48.80,11.34
Eintrag 2,Hamburg,53.33,10.00
Eintrag 3,Köln,45.75,-122.43
Eintrag 4,Frankfurt,47.61,-122.33
Eintrag 5,Duisburg,53.55,-113.49
Eintrag 6,Kassel,49.28,-123.13
Eintrag 7,Würzburg,51.05,-114.07


In [62]:
df

,Zeitstempel,Temperatur,Location,Luftdruck
0,2023-01-01 19:08,4.5,Berlin,1001.2
1,2023-01-01 18:17,6.3,München,997.8
2,2023-01-01 06:03,3.8,Wilhelmshaven,1002.5
3,2023-01-01 02:17,5.1,Kassel,1000.1
4,2023-01-01 22:02,4.9,Frankfurt,998.9
5,2023-01-01 16:00,5.7,Duisburg,1001.5
6,2023-01-01 21:04,4.2,Dresden,999.2
7,2023-01-01 11:16,6.0,Würzburg,1002.8


In [65]:
# Verwenden von right_on und left_on, um unterschiedliche
# Spaltennamen zu mergen
merged_df = df.merge(geo_df, left_on='Location', right_on='City', how='inner')
merged_df

,Zeitstempel,Temperatur,Location,Luftdruck,City,Breitengrad,Laengengrad
0,2023-01-01 19:08,4.5,Berlin,1001.2,Berlin,52.31,13.24
1,2023-01-01 18:17,6.3,München,997.8,München,48.80,11.34
2,2023-01-01 02:17,5.1,Kassel,1000.1,Kassel,49.28,-123.13
3,2023-01-01 22:02,4.9,Frankfurt,998.9,Frankfurt,47.61,-122.33
4,2023-01-01 16:00,5.7,Duisburg,1001.5,Duisburg,53.55,-113.49
5,2023-01-01 11:16,6.0,Würzburg,1002.8,Würzburg,51.05,-114.07


In [68]:
merged_df.drop('City', axis=1, inplace=True)
merged_df

,Zeitstempel,Temperatur,Location,Luftdruck,Breitengrad,Laengengrad
0,2023-01-01 19:08,4.5,Berlin,1001.2,52.31,13.24
1,2023-01-01 18:17,6.3,München,997.8,48.80,11.34
2,2023-01-01 02:17,5.1,Kassel,1000.1,49.28,-123.13
3,2023-01-01 22:02,4.9,Frankfurt,998.9,47.61,-122.33
4,2023-01-01 16:00,5.7,Duisburg,1001.5,53.55,-113.49
5,2023-01-01 11:16,6.0,Würzburg,1002.8,51.05,-114.07


#### Übungsaufgabe `merge`

Nachfolgender Dictionaries enthalten Daten zu Kunden und Produktkäufen.
Deine Aufgabe ist es, daraus zwei DataFrames zu erstellen und danach die beiden DataFrames so mitttels merge zu verbinden, dass zu allen Produktdaten die entsprechenden Kundendaten erscheinen, soweit verfügbar (ansonsten NaN-Werte).

In [70]:
customer_data = {
    'CustomerID': [101, 102, 103, 104, 105, 106, 107],
    'CustomerName': ['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank', 'Grace'],
    'Email': ['alice@mail.com', 'bob@mail.com', 'charlie@mail.com', 'david@mail.com', 'eva@mail.com', 'frank@mail.com', 'grace@mail.com'],
    'JoinDate': ['2022-05-01', '2021-06-15', '2020-08-20', '2022-11-25', '2023-01-05', '2021-09-10', '2020-12-31']
}

purchase_data = {
    'ClientID': [101, 102, 103, 108, 105, 106, 107, 102],
    'ProductID': [201, 202, 203, 204, 205, 206, 207, 205],
    'PurchaseDate': ['2023-01-10', '2023-02-15', '2023-01-20', '2023-03-10', '2023-01-30', '2023-03-05', '2023-01-25', '2023-04-01'],
}

In [71]:
customers = pd.DataFrame(customer_data)
purchases = pd.DataFrame(purchase_data)

In [72]:
purchases.merge(customers, left_on='ClientID', right_on='CustomerID', how='left')

,ClientID,ProductID,PurchaseDate,CustomerID,CustomerName,Email,JoinDate
0,101,201,2023-01-10,101.0,Alice,alice@mail.com,2022-05-01
1,102,202,2023-02-15,102.0,Bob,bob@mail.com,2021-06-15
2,103,203,2023-01-20,103.0,Charlie,charlie@mail.com,2020-08-20
3,108,204,2023-03-10,NaN,NaN,NaN,NaN
4,105,205,2023-01-30,105.0,Eva,eva@mail.com,2023-01-05
5,106,206,2023-03-05,106.0,Frank,frank@mail.com,2021-09-10
6,107,207,2023-01-25,107.0,Grace,grace@mail.com,2020-12-31
7,102,205,2023-04-01,102.0,Bob,bob@mail.com,2021-06-15
